### Twitter Scraper

Imports

In [2]:
from requests_oauthlib import OAuth1Session
import pandas as pd
import time
import json

Twitter API keys

In [3]:
consumer_key = 'Cd1NbKlV8xMLFQdZcUfh67CKA'
consumer_secret = 'M2r3BTi4glHD4QVOH6bqox0jNfnq3GKpl6CI98517B09Srzb1e'
access_key = '1219450490-BEuwysgyp8qFk7gNZX9A8DtTbhuIlO33ec1CvcP'
access_secret = 'Rg21ZrpZClgzk79ABxAG73nxNlha2e6EoaxPZoUj1X05I'

Helper functions

In [ ]:
class Tweet:

    def __init__(self, date, text, id, author):
        self.date = date
        self.text = text
        self.id = id
        self.author = author

    def dump(self):
        return {'date': self.date, 'text': self.text, 'id': self.id, 'author': self.author}

In [5]:
def get_url(hashtags, tweet_mode_extended=True, lang='en', links=True, retweets=True, replies=True, num=100):
    """
    This function puts together the url that is passed on to the twitter API.
    :param hashtags: list of hashtags
    :param tweet_mode_extended: If true, the API delivers full texts
    :param lang: English is the default language
    :param links: tweets with links are not included by default
    :param retweets: tweets that are retweets are not included by default
    :param replies: tweets that are replies are not included by default
    :param num: int, number of tweets queried for
    :return:
    """
    # Standard beginning of url
    api_plan_standard = 'https://api.twitter.com/1.1/search/tweets.json?'

    # Enable extended tweets
    if tweet_mode_extended is True:
        tweet_mode = 'tweet_mode=extended&'
    else:
        tweet_mode = ''

    # Build hashtag query
    q = 'q='
    if len(hashtags) > 1:
        q += '%28'
    for i in range(0, len(hashtags) - 1):
        q += ('%23' + hashtags[i] + '%20OR%20')
    q += ('%23' + hashtags[-1])
    if len(hashtags) > 1:
        q += '%29'

    # Exclude links
    if links is True:
        links = '%20-filter%3Alinks'
    else:
        links = ''

    # Exclude retweets
    if retweets is True:
        retweets = '%20-filter%3Aretweets'
    else:
        retweets = ''

    # Exclude replies
    if replies is True:
        replies = '%20-filter%3Areplies'
    else:
        replies = ''

    # assemble URL
    url = api_plan_standard + tweet_mode + q + '%20lang%3A' + lang + links + retweets + replies + '&count=' + str(num)

    return url

URL generieren

In [6]:
url = get_url(['corona', 'covid', 'coronavirus', 'covid19'])

Session authentifizieren

In [7]:
mytwitter = OAuth1Session(consumer_key, consumer_secret, access_key, access_secret)

File einrichten

In [8]:
file_name = 'tweets.csv'
tweet_bag = pd.DataFrame(columns=['date', 'text', 'id', 'author'])
tweet_bag.to_csv(file_name, sep='|', mode='a', index=False, encoding='UTF-8')

Tweets sammeln

In [ ]:
counter = 0
while True:
    try:
        response = mytwitter.get(url)
        response_data = json.loads(response.text)
        for item in response_data['statuses']:
            tweet = Tweet(item['created_at'], item['full_text'].replace('\n', ' '), item['id'], item['user']['screen_name'])
            tweet_bag = tweet_bag.append(tweet.dump(), ignore_index=True)
        tweet_bag.to_csv(file_name, sep='|', header=False, mode='a', index=False, encoding='UTF-8')
        #print('sleep for 5s') -> not necessary as RaspberryPi takes 1 second to write to the csv file
        #time.sleep(5) -> twitter API rate limits allow for continuous request every 5 seconds (of 100 tweets)
        print(str(counter+1) + '/10')
        counter += 1
        if counter == 10:
            print('drop dupes')
            drop_dupe = pd.read_csv(file_name, sep='|')
            drop_dupe = drop_dupe.drop_duplicates()
            drop_dupe = drop_dupe.sort_values(by='date')
            drop_dupe.to_csv(file_name, sep='|', mode='w', index=False, encoding='UTF-8')
            drop_dupe.to_csv('backup14.csv', sep='|', mode='w', index=False, encoding='UTF-8')
            counter = 0
    except KeyError:
        print(time.ctime(time.time()))
        print('rate limit: wait for 60s')
        time.sleep(60)
        counter = 0
    except json.decoder.JSONDecodeError:
        print(time.ctime(time.time()))
        print('twitter is not responding: wait for 60s')
        time.sleep(60)
        counter = 0
    except:
        print(time.ctime(time.time()))
        print('other error')
        counter = 0